In [1]:
import pandas as pd

In [4]:
# load hcp json file using json 
import json 
hcp_dict = json.load(open('../data/nap_fr.json'))




In [ ]:
hcp_dict_formated = {}
for page in hcp_dict :
    print(page)
    page_occs = hcp_dict[page]
    for occ in page_occs :
        hcp_dict_formated[occ] = page_occs[occ]


In [11]:
# load esco occupations 
esco_df = pd.read_csv('../data/occupations_fr.csv')
esco_df

,conceptType,conceptUri,iscoGroup,preferredLabel,altLabels,hiddenLabels,status,modifiedDate,regulatedProfessionNote,scopeNote,definition,inScheme,description
0,Occupation,http://data.europa.eu/esco/occupation/00030d09...,2166,directeur technique/directrice technique,régisseuse\nrégisseur\ndirectrice technique\nr...,NaN,released,2016-07-05T13:58:41Z,http://data.europa.eu/esco/regulated-professio...,NaN,NaN,http://data.europa.eu/esco/concept-scheme/occu...,NaN
1,Occupation,http://data.europa.eu/esco/occupation/000e93a3...,8121,opérateur de tréfilerie/opératrice de tréfilerie,opérateur de tréfilerie\nopératrice de tréfilerie,NaN,released,2016-07-05T17:09:43Z,http://data.europa.eu/esco/regulated-professio...,NaN,NaN,http://data.europa.eu/esco/concept-scheme/memb...,NaN
2,Occupation,http://data.europa.eu/esco/occupation/0019b951...,7543,contrôleur qualité des instruments de précisio...,contrôleuse qualité des instruments de précisi...,NaN,released,2016-07-06T09:21:20Z,http://data.europa.eu/esco/regulated-professio...,NaN,NaN,http://data.europa.eu/esco/concept-scheme/occu...,NaN
3,Occupation,http://data.europa.eu/esco/occupation/0022f466...,3155,technicien de la sécurité du trafic aérien/tec...,technicienne de la sécurité du trafic aérien\n...,NaN,released,2017-01-17T11:40:37Z,http://data.europa.eu/esco/regulated-professio...,NaN,NaN,http://data.europa.eu/esco/concept-scheme/memb...,NaN
4,Occupation,http://data.europa.eu/esco/occupation/002da35b...,2431,responsable yield manager,responsable des revenus hôteliers\nyield manag...,NaN,released,2017-01-17T13:33:42Z,http://data.europa.eu/esco/regulated-professio...,NaN,NaN,http://data.europa.eu/esco/concept-scheme/memb...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2937,Occupation,http://data.europa.eu/esco/occupation/ff656b3a...,2120,démographe,NaN,NaN,released,2016-07-05T16:21:57Z,http://data.europa.eu/esco/regulated-professio...,NaN,NaN,http://data.europa.eu/esco/concept-scheme/occu...,NaN
2938,Occupation,http://data.europa.eu/esco/occupation/ff8d4065...,9612,opérateur de tri/opératrice de tri,opérateur de tri\nopératrice de tri,NaN,released,2017-01-17T13:22:45Z,http://data.europa.eu/esco/regulated-professio...,NaN,NaN,http://data.europa.eu/esco/concept-scheme/memb...,NaN
2939,Occupation,http://data.europa.eu/esco/occupation/ffa4dd5d...,5414,convoyeur de fonds/convoyeuse de fonds,transporteur de fonds\ntransporteuse de fonds\...,NaN,released,2016-12-22T10:57:58Z,http://data.europa.eu/esco/regulated-professio...,NaN,NaN,http://data.europa.eu/esco/concept-scheme/memb...,NaN
2940,Occupation,http://data.europa.eu/esco/occupation/ffade2f4...,2422,adjoint administratif dans la fonction publiqu...,adjointe administrative dans la fonction publi...,NaN,released,2016-07-05T16:17:26Z,http://data.europa.eu/esco/regulated-professio...,NaN,NaN,http://data.europa.eu/esco/concept-scheme/occu...,NaN


In [86]:
import unidecode
import re
def clean_text(text):
    text = text.lower()
    # remove accents
    text = unidecode.unidecode(text)
    # remove / 
    text = text.replace('/', ' ')
    # remove , 
    text = text.replace(',', ' ')
    # remove special characters
    text = re.sub('[^a-zA-Z.\d\s]', '', text)
    # remove double spaces
    text = re.sub(' +', ' ', text)
  
    return text
 
def remove_sx_alt(text):
    return text.split('/')[0]



In [87]:
# prepare esco occupations
esco_corpus = esco_df['preferredLabel'].apply(remove_sx_alt).apply(clean_text).values
esco_ids = esco_df['preferredLabel'].values

# prepare hcp occupations
hcp_ids =list( hcp_dict_formated.keys())
hcp_corpus = [clean_text(hcp_dict_formated[occ_id]) for occ_id in hcp_dict_formated]

# stats 
print('hcp_corpus length : ', len(hcp_corpus))
print('esco_corpus length : ', len(esco_corpus))


hcp_corpus length :  1110
esco_corpus length :  2942


In [88]:
# tfidf char based 
# fit on esco  corpus
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(analyzer='char',ngram_range=(3,3), min_df=1, )
esco_tfid= vectorizer.fit_transform(esco_corpus)

features_names = vectorizer.get_feature_names_out()
# info 
print('features_names length : ', len(features_names))
print('esco space : ', esco_tfid.shape)

features_names length :  2498
esco space :  (2942, 2498)


In [95]:
import pprint

def get_top_matches(hcp_occ_query, n=10):
    """
    tfid : tfidf matrix 
    tfid_ids : tfidf matrix ids 
    n : number of top matches 
    """
    # trasform query
    hcp_occ_query_tfid = vectorizer.transform([hcp_occ_query])
    # cosine similarity
    cosine_similarities = esco_tfid.dot(hcp_occ_query_tfid.T).toarray().ravel()
    #print('cosine_similarities : ', cosine_similarities.toarray())
    # sort
    top_matches_indices = cosine_similarities.argsort()[::-1]
    top_matches_scores = cosine_similarities[top_matches_indices]

    # get top matches
    top_matches_ids = [(esco_ids[i],top_matches_scores[i] )for i in top_matches_indices[:n]]
    #top_matches_labels = [hcp_dict_formated[i] for i in top_matches_ids]
    return top_matches_ids

# test
key = '2729'
idx = hcp_ids.index(key)
hcp_occ =list(hcp_dict_formated.items())[idx]
jcp_occ_id , hcp_occ_query = hcp_occ


print(f'top_matches_ids for :\n {jcp_occ_id} :  {hcp_occ_query} ')
print('--'*30)
hcp_occ_query = clean_text(hcp_occ_query)

top_matches_ids = get_top_matches(hcp_occ_query)
print( top_matches_ids)


top_matches_ids for :
 2729 :  Autres professions intermédiaires artistiques et du spectacle n.c.a 
------------------------------------------------------------
[('artificier spectacles pyrotechniques/artificière spectacles pyrotechniques', 0.019707270984267375), ('directeur de production de spectacles/directrice de production de spectacles', 0.19017438615267074), ('professeur d’enseignements artistiques/professeure d’enseignements artistiques', 0.033076708508418186), ('directeur de salle de spectacles/directrice de salle de spectacles', 0.05082350390049898), ('directeur artistique/directrice artistique', 0.0), ('coiffeur spectacle/coiffeuse spectacle', 0.0), ('professeur de gestion des entreprises de la voie professionnelle/professeure de gestion des entreprises de la voie professionnelle', 0.07327166592174905), ('intervenant en prévention des risques professionnels/intervenante en prévention des risques professionnels', 0.03783143244899577), ('ouvreur de salle de spectacles/ouvreuse 

In [96]:
# TODO
# add alt labels
# control rules (atleast one token)
